IGENIERÍA DE DATOS

INGESTA DE DATOS

In [3]:
#Importo la librería pandas y gzip (cabe aclarar que se debe tener instalado pandas previamente, para eso utilizo el comando pip install pandas)
import pandas as pd
import gzip
import numpy as np

Decido utilizar los 3 archivos directamente en formato comprimido, para el caso del archivo "steam_games", se puede leer directamente utilizando
gzip pero para los siguientes 2 archivos: "user_reviews" y "user_items" hubo que hacer una función ya que el json venía con comillas simples

Ingesta de datos del primer archivo "steam_games"

In [5]:
#con gzip abrimos el json y generamos un dataframe
with gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/steam_games.json.gz', 'rt', encoding='utf-8') as steam_games:
       
    df_steam_games = pd.read_json(steam_games, lines=True)

df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,evcentric,7.656120e+16,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0
3,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,Riot-Punch,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328.0
4,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,doctr,7.656120e+16,"[{'item_id': '300', 'item_name': 'Day of Defea...",541.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS""",None,NaN,None,NaN
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada,None,NaN,None,NaN
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,None,NaN,None,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",None,NaN,None,NaN


In [7]:
steam_games = df_steam_games.drop(columns = ['items','user_id','steam_id','items_count','tags'])
steam_games

,publisher,genres,app_name,title,url,release_date,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None
1,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None
2,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None
3,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None
4,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns"


In [8]:
steam_games.dropna(inplace = True, how='all')

In [9]:
steam_games

,publisher,genres,app_name,title,url,release_date,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,None,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,http://steamcommunity.com/app/643980/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,None,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,http://steamcommunity.com/app/670290/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,None,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,http://steamcommunity.com/app/767400/reviews/?...,0.83,[Single-player],0.99,0.0,767400.0,None,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,http://steamcommunity.com/app/773570/reviews/?...,1.79,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns"


In [10]:
#realizo una función para separar la columna "release_date" en year, month y day para luego consumir en las funciones
def procesar_fecha(fecha):
    if fecha and fecha != 'none':
        partes = fecha.split("-")
        if len(partes) == 3:
            año = partes[0]
            mes = partes[1]
            dia = partes[2]
            return año, mes, dia
    return None, None, None

# Aplicp la función a la columna "release_date" y creo columnas separadas para año, mes y día
steam_games[['release_year', 'release_month', 'release_day']] = steam_games['release_date'].apply(lambda x: pd.Series(procesar_fecha(x)))

# Elimino la columna original "release_date"
steam_games.drop(columns=['release_date'], inplace=True)


In [11]:
steam_games

,publisher,genres,app_name,title,url,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,release_year,release_month,release_day
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,None,Kotoshiro,2018,01,04
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,None,Secret Level SRL,2018,01,04
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,None,Poolians.com,2017,07,24
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...,0.83,[Single-player],0.99,0.0,767400.0,None,彼岸领域,2017,12,07
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...,1.79,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS""",2018,01,04
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada,2018,01,04
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,2018,01,04
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",2017,09,02


Ingesta de datos del archivo "user_reviews" obteniendo un df_reviews_desanidado

In [13]:
#Ahora genero un dataframe para "user_reviews" utilizando una función que recorre y completa una lista vacía con la cual genero el DF
import ast #importo la librería ast a utilizar

info = [] 

for i in gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/user_reviews.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_user_reviews = pd.DataFrame(info) 
  
df_user_reviews


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [14]:
# Con las funciones explode y normalize desanidamos las columna "reviews" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_reviews_exploded = df_user_reviews.explode('reviews')
df_reviews_desanidado = pd.json_normalize(df_reviews_exploded['reviews'].dropna())

# Por otra parte reindexamos los DF
df_reviews_desanidado.reset_index(inplace=True)
df_reviews_exploded.reset_index(inplace=True)

# Concatenamos y eliminamosla columna que originalmente desanidamos

user_reviews = pd.concat([df_reviews_exploded,df_reviews_desanidado], axis=1)
user_reviews = user_reviews.drop(columns = ['reviews'])
user_reviews

,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,1,js41637,http://steamcommunity.com/id/js41637,3.0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,1,js41637,http://steamcommunity.com/id/js41637,4.0,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...,...,...
59328,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59329,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59330,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59331,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#verifico y borro los duplicados
user_reviews_duplicados =  user_reviews.duplicated( keep="first")
user_reviews_duplicados.sum()

10

In [16]:
user_reviews = user_reviews.drop_duplicates(keep = "first")

In [17]:
#realizo una función para separar la columna "posted" en dia, mes y año para luego consumir en las funciones

def procesar_fecha(data):
    import re
    if isinstance(data, str):
        match = re.search(r'(\w+) (\d+), (\d+)', data)
        if match:
            mes = match.group(1)
            dia = match.group(2)
            año = match.group(3)
            meses = {
                'January': '01',
                'February': '02',
                'March': '03',
                'April': '04',
                'May': '05',
                'June': '06',
                'July': '07',
                'August': '08',
                'September': '09',
                'October': '10',
                'November': '11',
                'December': '12'
            }
            mes_numero = meses.get(mes)
            if mes_numero is not None:
                return mes_numero, dia, año
    return None, None, None

# Aplico la función a la columna "posted" después de convertirla en cadenas de texto y unifico la fecha en el formato
#YYYY-MM-DD para consumir en las funciones
user_reviews[['Mes', 'Día', 'Año']] = user_reviews['posted'].astype(str).apply(lambda x: pd.Series(procesar_fecha(x)))
user_reviews['Año'].fillna(0, inplace=True)
user_reviews['Mes'].fillna(0, inplace=True)
user_reviews['Día'].fillna(0, inplace=True)


# Convierte las columnas 'Año', 'Mes' y 'Día' en enteros
user_reviews['Año'] = user_reviews['Año'].astype(int)
user_reviews['Mes'] = user_reviews['Mes'].astype(int)
user_reviews['Día'] = user_reviews['Día'].astype(int)
# Define una fecha predeterminada para las filas con valores cero
fecha_predeterminada = pd.to_datetime('1900-01-01')

# Crea una nueva columna 'fecha_review' con la fecha predeterminada para las filas con valores cero
user_reviews['fecha_review'] = user_reviews.apply(lambda row: pd.to_datetime(f"{row['Año']}-{row['Mes']}-{row['Día']}", format='%Y-%m-%d', errors='coerce') if row['Año'] != 0 and row['Mes'] != 0 and row['Día'] != 0 else fecha_predeterminada, axis=1)

# Elimino la columna original "posted"
user_reviews.drop(columns=['posted'], inplace=True)


/var/folders/_x/y99l7l9s60q1726y6701xbkw0000gn/T/ipykernel_85669/1260644294.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_reviews[['Mes', 'Día', 'Año']] = user_reviews['posted'].astype(str).apply(lambda x: pd.Series(procesar_fecha(x)))
/var/folders/_x/y99l7l9s60q1726y6701xbkw0000gn/T/ipykernel_85669/1260644294.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_reviews[['Mes', 'Día', 'Año']] = user_reviews['posted'].astype(str).apply(lambda x: pd.Series(procesar_fecha(x)))
/var/folders/_x/

In [18]:
user_reviews

,index,user_id,user_url,index,funny,last_edited,item_id,helpful,recommend,review,Mes,Día,Año,fecha_review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,11,5,2011,2011-11-05
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,,22200,No ratings yet,True,It's unique and worth a playthrough.,7,15,2011,2011-07-15
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,4,21,2011,2011-04-21
3,1,js41637,http://steamcommunity.com/id/js41637,3.0,,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,6,24,2014,2014-06-24
4,1,js41637,http://steamcommunity.com/id/js41637,4.0,,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,9,8,2013,2013-09-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59323,25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1900-01-01
59324,25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1900-01-01
59325,25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1900-01-01
59326,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1900-01-01


Ahora voy a realizar el análisis de sentimiento con NLP utilizando la librería NLTK

In [19]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Al ejecutar estos comandos, me encontré con errores por los certificados SSL, y tuve que instalar de otra forma como
muestro en el siguiente Markdown
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [20]:
#defino la función para analizar los sentimientos
def analizar_sentimiento(texto):
    if isinstance(texto, str):
        sia = SentimentIntensityAnalyzer()
        sentimiento = sia.polarity_scores(texto)
    
        # Determinar la etiqueta en función de la puntuación compuesta
        if sentimiento['compound'] >= 0.05:
            return '2'
        elif sentimiento['compound'] <= -0.05:
            return '0'
        else:
            return '1'
    else:
            return '1'

In [21]:
user_reviews['sentimiento'] = user_reviews['review'].apply(analizar_sentimiento)

/var/folders/_x/y99l7l9s60q1726y6701xbkw0000gn/T/ipykernel_85669/3754471914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_reviews['sentimiento'] = user_reviews['review'].apply(analizar_sentimiento)


In [22]:
#Borro registros con valores nulos en la columna "review" y duplicados
user_reviews = user_reviews.dropna(subset=['review'])
user_reviews = user_reviews.drop_duplicates( keep = "first")

In [23]:
#Elimino la columma "index"
user_reviews = user_reviews.drop(columns = ['index'])

Ingesta de datos del archivo "user_items" obteniendo un df_items_desanidado

In [25]:
#Ahora genero un dataframe para "users_items" utilizando una función que recorre y completa una lista vacía con la cual genero el DF
import ast #importo la librería ast a utilizar

info = [] 

for i in gzip.open('/Users/gaston/Documents/Carrera Data Analytics/Proyectos Individuales/Proyecto 1/users_items.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_users_items = pd.DataFrame(info) 
  
df_users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [26]:
# Con las funciones explode y normalize desanidamos las columna "items" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_items_exploded = df_users_items.explode('items')
df_items_desanidado = pd.json_normalize(df_items_exploded['items'])

# Por otra parte reindexamos los DF
df_items_exploded.reset_index(inplace=True)
df_items_desanidado.reset_index(inplace=True)

# Concatenamos y eliminamos la columna que previamente desanidamos
user_items = pd.concat([df_items_exploded,df_items_desanidado], axis=1)
user_items = user_items.drop(columns = ['items'])
user_items

,index,user_id,items_count,steam_id,user_url,index,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,10,Counter-Strike,6.0,0.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,20,Team Fortress Classic,0.0,0.0
2,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,30,Day of Defeat,7.0,0.0
3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,40,Deathmatch Classic,0.0,0.0
4,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5170010,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170010,373330,All Is Dust,0.0,0.0
5170011,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170013,519140,Minds Eyes,3.0,3.0


In [27]:
#identifico valores duplicados, como no hay registros duplicados, no es necesario borrar
user_items_duplicados =  user_items.duplicated( keep="first")
user_items_duplicados.sum()

0

In [28]:
#borro los duplicados
user_items = user_items.drop_duplicates(keep = "first")

Uno los datasets (DataFrames) user_items & steam_games ya que su info es complementaria y necesaria para las funciones
que debemos armar pero primero cambio el tipo de dato a float para poder unirlos

In [29]:
user_items['item_id']=user_items['item_id'].astype(float)

In [30]:
steam_games['item_id']=user_items['item_id'].astype(float)

In [31]:
user_items_steam_games = user_items.merge(steam_games, left_on='item_id',right_on='id',how='left')

In [32]:
user_items_steam_games

,index,user_id,items_count,steam_id,user_url,index,item_id_x,item_name,playtime_forever,playtime_2weeks,...,specs,price,early_access,id,metascore,developer,release_year,release_month,release_day,item_id_y
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,10.0,Counter-Strike,6.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,0.0,10.0,88,Valve,2000,11,01,40800.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,20.0,Team Fortress Classic,0.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,20.0,None,Valve,1999,04,01,550.0
2,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,30.0,Day of Defeat,7.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,30.0,79,Valve,2003,05,01,218060.0
3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,40.0,Deathmatch Classic,0.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,40.0,None,Valve,2001,06,01,105600.0
4,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,50.0,Half-Life: Opposing Force,0.0,0.0,...,"[Single-player, Multi-player, Valve Anti-Cheat...",4.99,0.0,50.0,None,Gearbox Software,1999,11,01,223530.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186816,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170011,388490.0,One Way To Die: Steam Edition,3.0,3.0,...,"[Single-player, Steam Achievements, Steam Trad...",Free to Play,0.0,388490.0,None,CoaguCo Industries,2015,09,01,219740.0
5186817,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170012,521570.0,You Have 10 Seconds 2,4.0,4.0,...,"[Single-player, Partial Controller Support, In...",Free To Play,0.0,521570.0,None,Tamationgames,2016,08,24,207250.0
5186818,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170013,519140.0,Minds Eyes,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5186819,88309,edward_tremethick,0,76561198331598578,http://steamcommunity.com/id/edward_tremethick,5170014,NaN,NaN,NaN,NaN,...,None,19.99,0.0,NaN,None,None,None,None,None,209080.0


Para la columna precio, defino que si el valor no es numérico, sea 0 para evitar errores de cálculos

In [33]:
user_items_steam_games['price'] = user_items_steam_games['price'].apply(lambda x: 0 if isinstance(x, str) else (x if pd.notna(x) else np.nan))


In [34]:
user_items_steam_games

,index,user_id,items_count,steam_id,user_url,index,item_id_x,item_name,playtime_forever,playtime_2weeks,...,specs,price,early_access,id,metascore,developer,release_year,release_month,release_day,item_id_y
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,10.0,Counter-Strike,6.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",9.99,0.0,10.0,88,Valve,2000,11,01,40800.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,20.0,Team Fortress Classic,0.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,20.0,None,Valve,1999,04,01,550.0
2,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,30.0,Day of Defeat,7.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,30.0,79,Valve,2003,05,01,218060.0
3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,40.0,Deathmatch Classic,0.0,0.0,...,"[Multi-player, Valve Anti-Cheat enabled]",4.99,0.0,40.0,None,Valve,2001,06,01,105600.0
4,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,50.0,Half-Life: Opposing Force,0.0,0.0,...,"[Single-player, Multi-player, Valve Anti-Cheat...",4.99,0.0,50.0,None,Gearbox Software,1999,11,01,223530.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186816,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170011,388490.0,One Way To Die: Steam Edition,3.0,3.0,...,"[Single-player, Steam Achievements, Steam Trad...",0.00,0.0,388490.0,None,CoaguCo Industries,2015,09,01,219740.0
5186817,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170012,521570.0,You Have 10 Seconds 2,4.0,4.0,...,"[Single-player, Partial Controller Support, In...",0.00,0.0,521570.0,None,Tamationgames,2016,08,24,207250.0
5186818,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170013,519140.0,Minds Eyes,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5186819,88309,edward_tremethick,0,76561198331598578,http://steamcommunity.com/id/edward_tremethick,5170014,NaN,NaN,NaN,NaN,...,None,19.99,0.0,NaN,None,None,None,None,None,209080.0


TRATAMIENTO DE DATOS

In [35]:
#borro las columnas "index" del dataframe user_items_steam_games
user_items_steam_games = user_items_steam_games.drop(columns = ['index'])

In [36]:
#Al intentar exportar el df a parquet, encontré que en la columna "metascore" tenía valores "none" que
#me generaban error ya que no es un Nan, por lo que decido reemplazarlos por NA

# Reemplazo 'NA' con NaN en la columna 'metascore'
user_items_steam_games['metascore'] = user_items_steam_games['metascore'].replace('NA', np.nan)

In [38]:
user_items_steam_games

,user_id,items_count,steam_id,user_url,item_id_x,item_name,playtime_forever,genres,app_name,title,...,discount_price,price,early_access,id,metascore,developer,release_year,release_month,release_day,item_id_y
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,[Action],Counter-Strike,Counter-Strike,...,NaN,9.99,0.0,10.0,88.0,Valve,2000,11,01,40800.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20.0,Team Fortress Classic,0.0,[Action],Team Fortress Classic,Team Fortress Classic,...,NaN,4.99,0.0,20.0,NaN,Valve,1999,04,01,550.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.0,Day of Defeat,7.0,[Action],Day of Defeat,Day of Defeat,...,NaN,4.99,0.0,30.0,79.0,Valve,2003,05,01,218060.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40.0,Deathmatch Classic,0.0,[Action],Deathmatch Classic,Deathmatch Classic,...,NaN,4.99,0.0,40.0,NaN,Valve,2001,06,01,105600.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50.0,Half-Life: Opposing Force,0.0,[Action],Half-Life: Opposing Force,Half-Life: Opposing Force,...,NaN,4.99,0.0,50.0,NaN,Gearbox Software,1999,11,01,223530.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186816,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490.0,One Way To Die: Steam Edition,3.0,"[Adventure, Free to Play]",One Way To Die: Steam Edition,One Way To Die: Steam Edition,...,NaN,0.00,0.0,388490.0,NaN,CoaguCo Industries,2015,09,01,219740.0
5186817,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570.0,You Have 10 Seconds 2,4.0,"[Casual, Free to Play, Indie]",You Have 10 Seconds 2,You Have 10 Seconds 2,...,NaN,0.00,0.0,521570.0,NaN,Tamationgames,2016,08,24,207250.0
5186818,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140.0,Minds Eyes,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5186819,edward_tremethick,0,76561198331598578,http://steamcommunity.com/id/edward_tremethick,NaN,NaN,NaN,None,None,None,...,14.99,19.99,0.0,NaN,NaN,None,None,None,None,209080.0


In [37]:
#Decido borrar las columnas playtime_2weeks, specs y publisher para reducir espacio en el dataframe user_items_steam_games
user_items_steam_games = user_items_steam_games.drop(columns = ['playtime_2weeks'])
user_items_steam_games = user_items_steam_games.drop(columns = ['specs'])
user_items_steam_games = user_items_steam_games.drop(columns = ['publisher'])

Luego de confeccionados los dataframes con la info que necesito para consumir en las funciones,
decido guardarlos en archivos parquet y no en csv para optimizar 

Para eso primero instalo la biblioteca pyarrow con el comando pip install pyarrow

In [ ]:
#importo pyarrow
import pyarrow as pa

In [ ]:
user_reviews.to_parquet('user_reviews.parquet', engine='pyarrow', compression='snappy')
user_items_steam_games.to_parquet('users_games.parquet', engine='pyarrow', compression='snappy')


Para la API tuve que instalar fastapi y uvicorn para poder crear el servidor web (consultar documentación en https://fastapi.tiangolo.com)